# Lab3 exercises 1

### Intro
In this laboratory, you will learn more about frequent itemsets and association rules. You will first use an existing library that implements Apriori and FP-Growth. Then, you will implement your own version of Apriori.

#### Preliminary steps

In [1]:
!test -e online_retail.csv || wget https://github.com/dbdmg/data-science-lab/raw/master/datasets/online_retail.csv -O online_retail.csv

#### Exercise 2.1.1
This exercise will work on the Online Retail Data Set. In particular, you will do some data preprocessing on the dataset to extract all itemsets available (where each itemset is the collection of items contained in a single invoice). Then, using FP-Growth and Apriori implementations, you will extract a list of frequent itemsets. From those, you will finally extract several different association rules.

First, you need to load the dataset into memory, using the csv module. Make sure you identify all valid rows. Also consider that rows having an InvoiceNo that starts with C should be discarded, as they indicate that the invoice is about a cancelled purchase.

In [2]:
import csv

dataset = [ [], [], [], [], [], [], [], []]
NUMBER_OF_COLS = 8
NUMBER_OF_ROWS = 0
MEASUREMENTS = []
with open("online_retail.csv") as f:
    for i, row in enumerate(csv.reader(f)):
        if i == 0:
            MEASUREMENTS = row
            continue
        if len(row) == NUMBER_OF_COLS:  # only do this if the number of columns is as expected
            if not row[0].startswith("C"):  # discard invoice canceled
                NUMBER_OF_ROWS = NUMBER_OF_ROWS + 1
                dataset[0].append(row[0])
                dataset[1].append(row[1])
                dataset[2].append(row[2])
                dataset[3].append(row[3])
                dataset[4].append(row[4])
                dataset[5].append(row[5])
                dataset[6].append(row[6])
                dataset[7].append(row[7])

print(f"Measurements {MEASUREMENTS}")
print(f"Loaded {NUMBER_OF_ROWS} rows")

Measurements ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']
Loaded 532621 rows


In [3]:
idx = 2
out = [f"{MEASUREMENTS[i]}={dataset[i][idx]}" for i in range(NUMBER_OF_COLS)]
print(out)

['InvoiceNo=536365', 'StockCode=84406B', 'Description=CREAM CUPID HEARTS COAT HANGER', 'Quantity=8', 'InvoiceDate=12/1/2010 8:26', 'UnitPrice=2.75', 'CustomerID=17850', 'Country=United Kingdom']


#### Exercise 2.1.2
Now that you have a dataset of items, you should aggregate it at an “invoice” level. For each invoice (identified by InvoiceNo) there can be multiple items (from multiple rows) in the dataset. For each invoice, you should build a list of all items belonging to it. For the example invoice presented in 1.2.1, you want to build the following list:

In [4]:
# Method that use list. Better to use set(). List search is O(n); set search is O(1)
# invoices = {}
# for i in range(NUMBER_OF_ROWS):
#    el = invoices.get(dataset[0][i], [])
#    el.append(dataset[2][i])
#    invoices[dataset[0][i]] = el

In [5]:
# invoices

In [6]:
invoices = {}
for i in range(NUMBER_OF_ROWS):
    if dataset[0][i] not in invoices:
        invoices[dataset[0][i]] = set()
    invoices[dataset[0][i]].add(dataset[2][i])

In [7]:
# invoices

In [8]:
invoices_codes = list(invoices.keys())
invoices_list = list(invoices.values())
len(invoices_codes), len(invoices_list), invoices_list[0]

(22064,
 22064,
 {'CREAM CUPID HEARTS COAT HANGER',
  'GLASS STAR FROSTED T-LIGHT HOLDER',
  'KNITTED UNION FLAG HOT WATER BOTTLE',
  'RED WOOLLY HOTTIE WHITE HEART.',
  'SET 7 BABUSHKA NESTING BOXES',
  'WHITE HANGING HEART T-LIGHT HOLDER',
  'WHITE METAL LANTERN'})

In [9]:
invoices["574021"]

{'DOORMAT KEEP CALM AND COME IN',
 'GARDENERS KNEELING PAD KEEP CALM ',
 'HOT WATER BOTTLE KEEP CALM'}

#### Exercise 2.1.3
You should now have a list (one for each invoice) of lists (each list containing the items bought for that invoice). Now, we need to convert this into a matrix form. Of the many possible formats, we will use the one expected by the Mlxtend library, which is as follows. Given an ordered list of M possible items (in this case, all possible products that can be bought), and given N itemsets (in this case, invoices), we should build a matrix of N rows and M columns. The element at the ith row and jth column should be 1 if the ith itemset (invoice) contains the jth item (product), 0 otherwise.

In [10]:
# Running the example
import pandas as pd
all_items = ["a", "b", "c", "d"] # this is your list of items
pa_matrix = [
   [1,1,1,0],
   [0,1,1,0],
   [1,0,1,1],
   [1,1,0,0]
] # this is the matrix you built from the itemsets
dfExample = pd.DataFrame(data=pa_matrix, columns=all_items)
dfExample

,a,b,c,d
0,1,1,1,0
1,0,1,1,0
2,1,0,1,1
3,1,1,0,0


At this point, we need to build an NxM matrix. Here, N is the number of transactions (i.e. invoices) available, while M is the number of total (unique) items that can be in a transaction. The element at i-th row and j-th column will be 1 if the i-th transaction contains the j-th item, 0 otherwise.
First, we need to define a list of unique items. We can build a collection of unique items by resorting to sets. By iterating on each of the transactions and adding all items to the set, we can make sure to be including all possible items. Since we are using a set, we will not be concerned with duplicate entries.

In [11]:
all_items_set = set()
for items in invoices.values():
    all_items_set.update(items)

print(f"We have {len(all_items_set)} different items")

We have 4208 different items


Here, we make use of the update() method available for sets. As the name says, this method takes as input a collection of values and adds them to the set (being the destination a set, duplicates are discarded).
Now that we have a list of all possible values, we need to define a specific order among them. Since sets are unordered, we can convert the set into a list, and sort it (this makes the results repeatable).

In [12]:
all_items = sorted(list(all_items_set))

We can now build our matrix row by row. For each row (i.e. for each invoice, or transaction), we can build a row vector having 0 or 1 based on the presence of the specific item within the specific row. We can do this with a nested list comprehension, as follows.

In [13]:
def get_presence_matrix(invoices, all_items):
    item_pos_dict = { k: v for v, k in enumerate(all_items) }
    presence_matrix = []
    for invoice in invoices.values():
        row = [False] * len(all_items)  # Better with False instead of 0
        for item in invoice:
            row[item_pos_dict[item]] = True  # Better with True instead of 1
        presence_matrix.append(row)
    return presence_matrix

In [14]:
presence_matrix = get_presence_matrix(invoices, all_items)

In [15]:
df = pd.DataFrame(data=presence_matrix, columns=all_items)
df

,,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22059,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
22060,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
22061,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
22062,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


#### Exercise 2.1.4
With the df that you defined in the previous exercise, you can now use the fp_growth function. This function, which is described in the detail in the official documentation. The first argument required is the previously built DataFrame, df. The second is the minimum support (minsup), i.e. the minimum fraction of the entire dataset in which the itemset should show up for it to be considered “frequent”. Try using different values of minsup, such as 0.5, 0.1, 0.05, 0.02, 0.01. How many results do you obtain as minsup varies? You can check the number of frequent itemsets identified and print them all with the following code snipped:

In [16]:
# Help me: https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/fpgrowth/
from mlxtend.frequent_patterns import fpgrowth
fi = fpgrowth(dfExample, 0.05)
print(len(fi))
print(fi.to_string())

11
    support   itemsets
0      0.75        (2)
1      0.75        (1)
2      0.75        (0)
3      0.25        (3)
4      0.50     (1, 2)
5      0.50     (0, 1)
6      0.50     (0, 2)
7      0.25  (0, 1, 2)
8      0.25     (0, 3)
9      0.25     (2, 3)
10     0.25  (0, 2, 3)


/Users/alex/Documents/istruzione/Polito1/ds_lab/python-exercies-and-lab/labs/venv/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [17]:
from mlxtend.frequent_patterns import fpgrowth

for minsup in [ 0.5, 0.1, 0.05, 0.02, 0.01 ]:
    freq_itemsets = fpgrowth(df, minsup, use_colnames=True)
    print(f"{minsup} => {len(freq_itemsets)}")

0.5 => 0
0.1 => 1
0.05 => 23
0.02 => 303
0.01 => 1472


In [18]:
freq_itemsets

,support,itemsets
0,0.102429,(WHITE HANGING HEART T-LIGHT HOLDER)
1,0.020803,(KNITTED UNION FLAG HOT WATER BOTTLE)
2,0.019443,(RED WOOLLY HOTTIE WHITE HEART.)
3,0.017223,(SET 7 BABUSHKA NESTING BOXES)
4,0.013914,(WHITE METAL LANTERN)
...,...,...
1467,0.012373,"(JUMBO BAG VINTAGE DOILY , LUNCH BAG VINTAGE D..."
1468,0.010560,"(LUNCH BAG VINTAGE DOILY , LUNCH BAG BLACK SK..."
1469,0.010515,"(LUNCH BAG VINTAGE DOILY , LUNCH BAG APPLE DES..."
1470,0.010696,"(LOVE HOT WATER BOTTLE, HOT WATER BOTTLE KEEP ..."


#### Exercise 2.1.5
Consider the itemsets extracted for minsup = 0.02. How many items are contained? Which ones would you be considered the most useful?

In [19]:
freq_itemsets = fpgrowth(df, 0.02)
print(f"Items in the itemsets {len(freq_itemsets)}")
freq_itemsets[freq_itemsets["itemsets"].map(len) > 1]

Items in the itemsets 303


,support,itemsets
246,0.021392,"(1824, 1825)"
247,0.029007,"(162, 166)"
248,0.021302,"(165, 166)"
249,0.024429,"(3970, 3966)"
250,0.022435,"(3904, 2837)"
251,0.026242,"(1858, 2046)"
252,0.037391,"(1856, 1858)"
253,0.023341,"(1856, 1871)"
254,0.032814,"(1858, 1871)"
255,0.026514,"(1858, 1846)"


These are the itemsets having multiple items occurring together most often. This is already an actionable insight, since it provides information about what customers are interested in buying as a bundle. A simple recommendation system could use this information to suggest useful items to customers (e.g. the "frequently bought together" functionality on Amazon).

#### Exercise 2.1.6
Use the value returned by fpgrowth to extract the relevant association rules.

In [20]:
M = df.values # matrix from the df dataframe
support_2656 = len(M[M[:, 2656] == 1])/len(M)
support_1599 = len(M[M[:, 1599] == 1])/len(M)
support_both = len(M[(M[:, 2656] == 1) & (M[:, 1599] == 1)])/len(M)
print(f"Confidence 2656 => 1599: {support_both / support_2656}")
print(f"Confidence 1599 => 2656: {support_both / support_1599}")

Confidence 2656 => 1599: 0.8263707571801567
Confidence 1599 => 2656: 0.6236453201970443


#### Exercise 2.1.7
Extract the association rules from the frequent itemsets extracted with minsup = 0.01. You can find the documentation for association_rules() on the official documentation. You can use the confidence as the metric to identify the rules, and a minimum threshold of 0.85 (feel free to vary these values and observe how the results vary).

Now that we know how association rules can be generated and evaluated, we can use mlxtend's association_rules() function to extract all association rules from a list of frequent itemsets (extracted with minsup = 0.01). We can filter the extracted association rules using a minimum threshold on the confidence, support or lift.
The association_rules() function takes as first argument the output of fpgrowth() (i.e. a DataFrame with frequent itemsets and their support). The second argument is the matric that we want to use for filtering association rules. As recommended by the exercise, we are going to use the confidence. As threshold value, as recommended, we will be using 0.85. As a side exercise, you can tweak these parameters and observe how the results vary.

In [21]:
from mlxtend.frequent_patterns import association_rules

fi = fpgrowth(df, 0.01)
association_rules(fi, 'confidence', 0.85)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(3547, 723)",(2861),0.017177,0.046864,0.014775,0.860158,18.354481,0.013970,6.815824,0.962043
1,"(3547, 724, 723)",(2861),0.012282,0.046864,0.011104,0.904059,19.291256,0.010528,9.934613,0.959954
2,"(3547, 3981, 723)",(2861),0.011920,0.046864,0.010968,0.920152,19.634657,0.010409,11.936898,0.960519
3,"(3547, 724, 3981)",(2861),0.013733,0.046864,0.011784,0.858086,18.310257,0.011140,6.716286,0.958549
4,"(1856, 1869, 1871)",(1858),0.014005,0.094815,0.012146,0.867314,9.147426,0.010819,6.822003,0.903331
5,"(3338, 3292)",(3339),0.013416,0.023885,0.012011,0.895270,37.482435,0.011690,9.320323,0.986556
6,(1731),(1732),0.010877,0.010741,0.010016,0.920833,85.726864,0.009899,12.495897,0.999204
7,(1732),(1731),0.010741,0.010877,0.010016,0.932489,85.726864,0.009899,14.651378,0.999066
8,"(723, 724, 3981)",(2861),0.013234,0.046864,0.011376,0.859589,18.342333,0.010756,6.788190,0.958162
9,"(3561, 1858)",(1092),0.012781,0.032088,0.011285,0.882979,27.517009,0.010875,8.271244,0.976135


The rules returned by association_rules() are in the form (antecedents) => (consequents) and are extracted based on the considerations made for the previous exercise: we can use the previous code to compute the confidence for any of the presented association rules, and check whether we have a match with the results obtained. Consider the association rules 1731 => 1732 and 1732 => 1731.

In [22]:
M = df.values # matrix from the df dataframe
support_1731 = len(M[M[:, 1731] == 1])/len(M)
support_1732 = len(M[M[:, 1732] == 1])/len(M)
support_both = len(M[(M[:, 1731] == 1) & (M[:, 1732] == 1)])/len(M)
print(f"Confidence 1731 => 1732: {support_both / support_1731}")
print(f"Confidence 1732 => 1731: {support_both / support_1732}")

Confidence 1731 => 1732: 0.9208333333333334
Confidence 1732 => 1731: 0.9324894514767933


#### Exercise 2.1.8
(*) Rerun the experiments from point 4 with apriori() (documentation on the official website). Do the results match with the ones found by FP-Growth? Is Apriori faster or slower than FP-Growth?

In [23]:
from mlxtend.frequent_patterns import apriori
from timeit import timeit

print("FP-growth", timeit(lambda: fpgrowth(df, 0.01), number=1), "seconds")
print("Apriori", timeit(lambda: apriori(df, 0.01), number=1), "seconds")

FP-growth 2.111715125007322 seconds
Apriori 12.719232167000882 seconds


In [24]:
fi_apriori = apriori(df, 0.02)
fi_fpgrowth = fpgrowth(df, 0.02)
len(fi_apriori), len(fi_fpgrowth)

(303, 303)

We could have a first quantitative feedback on whether the two results are the same by observing the number of results we obtained. Indeed, both return 303 elements, meaning that the two algorithms are at least in accordance with the number of frequent itemsets that can be extracted. 